In [228]:
import numpy as np
import matplotlib.pyplot as pp
# import scipy.linalg as la
%matplotlib inline

data = np.loadtxt(open("RegressionData.txt","rb"), delimiter=" ", skiprows=0)
# print data

X = data[:,0]  # random numbers x_n drawn from uniform distribution over [0,1]
print X
Y = data[:,1]  # labels/target values t_n for x_n, generated using 2*pi*x_n, gaussian noise with standard deviation sigma=0.25
print Y

[ 0.5503  0.9206  0.5359  0.6081  0.0202  0.8545  0.2357  0.4847  0.3996
  0.1957]
[-0.5894 -0.2507 -0.0468 -0.3402  0.2857 -1.0683  0.8605 -0.0801  0.6837
  1.185 ]


# 3.1

In [330]:
def init(X, layers=[10, 3, 10]):
    rows = len(X)+1  # include bias node
    columns = len(layers)

    # activations, 3 columns: bias + input layer, 1 hidden layer, output layer
    S = np.zeros((rows, columns))
    
    # bias
    S[0,:] = -1.

    # connection weights W[from-layer][from-index, to-index]
    W = [np.random.uniform(-0.5, 0.5, rows*rows).reshape(rows, rows) for i in range(columns-1)]
#     W = [np.ones((rows, rows)) for i in range(columns-1)]
    for w in range(len(W)):
        left = layers[w]  # max from-index
        right = layers[w+1]  # max to-index
        
        for i in range(rows):
            for j in range(rows):
                # only allow weights between nodes that exist in the model
                # may not come from too high index
                # may not go to too high index
                # may not go to bias node
                if i > left or j > right or j == 0:
                    W[w][i,j] = 0.
#         print W[w], "W["+str(w)+"]"
    
    # transfer fct
    f = np.tanh
#     f = lambda x: x
    # 1st derivative of transfer fct
    fd = lambda h : (1./np.cosh(h))**2

    return S, W, f, fd

# calculate activations
def forward(S, W, X, f):
    rows, columns = S.shape
    
    # init
    H = np.zeros(S.shape)
    S[1:,0] = X.T
    H[1:,0] = X.T
    
    # propagation
    for v in range(1, columns):
        S[1:,v] = H[1:,v] = W[v-1].T.dot(S[:,v-1])[1:]
        if v < columns-1:
            S[1:,v] = f(H[1:,v])
    
    return H, S

# calculate output error as quadratic error
def output_error(Yt, Yx):  # target labels, computed labels
#     print Yt, len(Yt)
#     print Yx, len(Yx)
    return ((Yt-Yx)**2)/2.

# calculate local error for each training point
def backward(S, H, W, Eo, fd):
    rows, columns = S.shape
    
    # init
    D = np.zeros((rows, columns))
    D[1:,-1] = 1  # =1 if f = identity
    
    # propagate
    for v in range(columns-1)[::-1]:
        D[:,v] = fd(H[:,v]) * (D[:,v+1].T.dot(W[v]))
    
    return D

def gradient(S, D, Yt, Yx):
    rows, columns = S.shape
    # 1st factor as per script p. 17 (1.9)
    dedy = np.insert(Yx - Yt, 0, 0)  # row vector
#     print dedy, "dedy"
    
    # 2nd factor as per script p. 18 (1.11)
    dydw = [S[:,i+1,np.newaxis] * D[:,i,np.newaxis].T for i in range(columns-1)]
    # dydw should now be a quadratic matrix indexed as dydw[from,to]
#     print dydw, "dydw"
#     print dydw[0].shape, "dydw[0].shape"
    
    # script p. 17 (1.8), (1.7)
    N = float(len(Yt))
    G = [np.sum(np.array([dydw[i] * e for e in dedy]), 0)/N for i in range(len(dydw))]
#     print G[0].shape, "dedw[0].shape"
#     print G[0]
    
    return G

def weight_update(W, G, eta):
#     print W[0].shape
#     print G[0].shape
    
    for w in range(len(W)):
        W[w] = W[w] - eta*G[w]
    
    return W

def ex31(X, Yt, S, W, f, fd):
#     rows, columns = S.shape
#     W[0][1,1] = 1
#     W[1][1,3] = 1
    Ey_old = Yt*0
    i = 0
    while i < 3000:
        H, S = forward(S, W, X, f)
    #     print H, "total input of transfer fct: H"
#         print S, "activations S"

        Ey = output_error(Yt, S[1:,-1])
    #     print Eo, "output error"

        D = backward(S, H, W, Ey, fd)
#         print D, "local error D"

        G = gradient(S, D, Yt, S[1:,-1])
    #     print G[0], "gradient G"
    #     print G[0].shape, "gradient shape"

        eta = .5
        W = weight_update(W, G, eta)
        
        converged = np.all((np.abs(Ey - Ey_old)/Ey_old) < 10.**-5)
        if converged:
            break
        else:
            print "iteration", i, " error mean: ", np.mean(Ey)
            i += 1
            Ey_old = Ey

    for i in range(len(W)):
        print W[i]
    
    print S

# cache variables
_S, _W, _f, _fd = init(X)
ex31(X, Y, _S, _W, _f, _fd)

iteration 0  error mean:  0.335753303898
iteration 1  error mean:  0.355089008034
iteration 2  error mean:  0.377483075302
iteration 3  error mean:  0.401254172721
iteration 4  error mean:  0.425474649759
iteration 5  error mean:  0.449621750587
iteration 6  error mean:  0.473397654999
iteration 7  error mean:  0.496633662734
iteration 8  error mean:  0.519237335317
iteration 9  error mean:  0.541162308142
iteration 10  error mean:  0.562390496524
iteration 11  error mean:  0.582921313485
iteration 12  error mean:  0.602764981888
iteration 13  error mean:  0.621938306438
iteration 14  error mean:  0.640461961219
iteration 15  error mean:  0.658358731615
iteration 16  error mean:  0.675652368613
iteration 17  error mean:  0.692366842284
iteration 18  error mean:  0.708525858855
iteration 19  error mean:  0.724152553565
iteration 20  error mean:  0.739269301628
iteration 21  error mean:  0.75389760884
iteration 22  error mean:  0.768058055943
iteration 23  error mean:  0.781770279132
ite

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:126: RuntimeWarning: divide by zero encountered in divide
